In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Rutas
ruta_datos = '/content/drive/MyDrive/Laboratorio_3/data/sell-in.txt'
ruta_productos = '/content/drive/MyDrive/Laboratorio_3/data/tb_productos.txt'
ruta_stocks = '/content/drive/MyDrive/Laboratorio_3/data/tb_stocks.txt'
ruta_predecir = '/content/drive/MyDrive/Laboratorio_3/data/product_id_apredecir201912.txt'

#Leer archivos
df = pd.read_csv(ruta_datos, sep='\t', decimal='.')
df_productos = pd.read_csv(ruta_productos, sep='\t', decimal='.')
df_stocks = pd.read_csv(ruta_stocks, sep='\t', decimal='.')
df_predecir = pd.read_csv(ruta_predecir, sep='\t')

In [4]:
#Convertir 'periodo' de formato YYYYMM en df
df['periodo'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

#Crear columna periodo_ym
df['periodo_ym'] = df['periodo'].dt.strftime('%Y%m').astype(int)

## **TRIVIAL EVO**

In [27]:
#Filtrar febrero 2019
df_201902 = df[df['periodo_ym'] == 201902]

#Agrupar y sumar toneladas
df_tn = df_201902.groupby('product_id', as_index=False)['tn'].sum()

#Cruzar con lista de product_id a predecir
df_final = df_predecir.merge(df_tn, on='product_id', how='left')
df_final['tn'] = df_final['tn'].fillna(0.0)
df_final.head()

,product_id,tn
0,20001,1259.09363
1,20002,1043.01349
2,20003,758.32657
3,20004,441.70332
4,20005,409.89950


In [28]:
# Trivial Último Año

# Definir meses del año 2019
ultimos_12m = list(range(201901, 201913))  # de 201901 a 201912 inclusive
df_trim12 = df[df['periodo_ym'].isin(ultimos_12m)]

# Base completa: todos los product_id x meses del año
productos = df_predecir[['product_id']]
meses = pd.DataFrame({'periodo_ym': ultimos_12m})
base_completa = productos.merge(meses, how='cross')

# Merge con datos reales de tn
df_completo = base_completa.merge(
    df_trim12[['product_id', 'periodo_ym', 'tn']],
    on=['product_id', 'periodo_ym'],
    how='left'
)
df_completo['tn'] = df_completo['tn'].fillna(0.0)

# Calcular suma total del año (sin dividir por 12)
df_prom12 = df_completo.groupby('product_id', as_index=False)['tn'].sum()

# Ver resultados
df_prom12.head()


,product_id,tn
0,20001,17456.79264
1,20002,14105.24570
2,20003,9419.71689
3,20004,7526.58394
4,20005,8019.24125


In [29]:
# Agregar columna con tn de febrero 2019
df_final_feb = df_final[['product_id', 'tn']].rename(columns={'tn': 'tn_feb19'})
df_prom12 = df_prom12.merge(df_final_feb, on='product_id', how='left')

# Calcular ratio febrero / promedio anual
df_prom12['febrero_sobre_año'] = 1 - (df_prom12['tn_feb19'] / df_prom12['tn'])

# Ver resultados
df_prom12.head()



,product_id,tn,tn_feb19,febrero_sobre_año
0,20001,17456.79264,1259.09363,0.927874
1,20002,14105.24570,1043.01349,0.926055
2,20003,9419.71689,758.32657,0.919496
3,20004,7526.58394,441.70332,0.941314
4,20005,8019.24125,409.89950,0.948886


In [32]:
### Trivial Último Año Ajustado con factor por producto

# Filtrar meses 201901 a 201912
meses_12m = list(range(201901, 201913))
df_trim12 = df[df['periodo_ym'].isin(meses_12m)]

# Crear base completa: todos los product_id x 12 meses
productos = df_predecir[['product_id']]
meses = pd.DataFrame({'periodo_ym': meses_12m})
base_completa = productos.merge(meses, how='cross')

# Merge con tn y completar faltantes
df_completo = base_completa.merge(
    df_trim12[['product_id', 'periodo_ym', 'tn']],
    on=['product_id', 'periodo_ym'],
    how='left'
)
df_completo['tn'] = df_completo['tn'].fillna(0.0)

# Calcular promedio anual por producto
df_prom_ajustado = df_completo.groupby('product_id', as_index=False)['tn'].sum()
df_prom_ajustado['tn'] = df_prom_ajustado['tn'] / 12  # promedio mensual

# Merge con factores individuales calculados previamente (febrero_sobre_año)
# Asegurarse de tener el DataFrame con esos valores
df_prom_ajustado = df_prom_ajustado.merge(
    df_prom12[['product_id', 'febrero_sobre_año']],
    on='product_id',
    how='left'
)

# Aplicar el factor individual
df_prom_ajustado['tn'] = df_prom_ajustado['tn'] * df_prom_ajustado['febrero_sobre_año']

# Dejar solo product_id y tn ajustada
df_prom_ajustado = df_prom_ajustado[['product_id', 'tn']]

# Ver resultado final
df_prom_ajustado.head()

,product_id,tn
0,20001,1349.808251
1,20002,1088.519351
2,20003,721.782527
3,20004,590.406718
4,20005,634.111813


In [33]:
#Exportar
output_path = '/content/drive/MyDrive/Laboratorio_3/promedio_12m_ajustadoxitem.csv'
df_prom_ajustado.to_csv(output_path, index=False, sep=',', decimal='.')

#Confirmar
print(f"✅ Promedio ajustado exportado con {len(df_prom_ajustado)} registros.")
print(f"📁 Ruta: {output_path}")
df_prom_ajustado.head()

✅ Promedio ajustado exportado con 780 registros.
📁 Ruta: /content/drive/MyDrive/Laboratorio_3/promedio_12m_ajustado.csv


,product_id,tn
0,20001,1349.808251
1,20002,1088.519351
2,20003,721.782527
3,20004,590.406718
4,20005,634.111813
